In [1]:
using JuMP
using MosekTools, SCS
using LinearAlgebra
using ControlSystemsBase

In [53]:
A = [1 0.1; 0 1];
B = [0; 1];
sys = ss(A,B,I,0,1)
Q = I
R = I
Kd = lqr(sys,Q,R)
sys_c= d2c(sys)
Ac = sys_c.A
Bc = sys_c.B
Kc = lqr(sys_c,Q,R)
print(round.(Kc,digits=4))

[1.0 1.1466]

In [57]:
model = Model(Mosek.Optimizer);
n = 2
@variable(model, Σ[i=1:n, j=1:n], PSD)
@variable(model, Z[i=1:1, j=1:1], PSD)
@variable(model, Y[i=1:n, j=1:n], PSD)
@variable(model, W[i=1:1, j=1:1], PSD)
@variable(model, L[i=1:1, j=1:n])

@objective(model, Min, tr(Q*Σ)+tr(R*Z)+tr(Y)+5*tr(W))
@constraint(model, Σ-1e-5*I >= 0, PSDCone())
# Z >= KΓK'
@constraint(model, [Z L; L' Σ] >= 0, PSDCone())
@constraint(model, [Y I; I Σ] >= 0, PSDCone())
@constraint(model, [W K; K' I] >= 0, PSDCone())
# Lyupanov
#@constraint(model, Ac*Σ + Bc*L + (Ac*Σ + Bc*L)'+3I >= 0, PSDCone())
@constraint(model, Ac*Σ + Bc*L + (Ac*Σ + Bc*L)'+3I <= 0, PSDCone())
set_silent(model)
optimize!(model);
Σ, Z, L = value.(Σ), value.(Z), value.(L)
K₁ = L * inv(Σ) 

1×2 Matrix{Float64}:
 -0.910226  -1.03446

In [66]:
model = Model(Mosek.Optimizer);
n = 2
@variable(model, Σ[i=1:n, j=1:n], PSD)
@variable(model, Zᵢ[i=1:n, j=1:n], PSD)
@variable(model, Z[i=1:1, j=1:1], PSD)
@variable(model, L[i=1:1, j=1:n])

@objective(model, Min, tr(Q*Σ)+tr(R*Z)+10*tr(Zᵢ)+tr(L*ones(n,1)))
@constraint(model, Σ-1e-5*I >= 0, PSDCone())
# Z >= KΓK'
@constraint(model, [Z L; L' Σ] >= 0, PSDCone())
@constraint(model, [Zᵢ I; I Σ] >= 0, PSDCone())
# Lyupanov
@constraint(model, [Σ-A*Σ*A'-B*L*A'-A*L'*B'-3I B*L; L'*B' Σ]>=0, PSDCone())
set_silent(model)
optimize!(model);
Σ, Z, L = value.(Σ), value.(Z), value.(L)
K₂ = L * inv(Σ)
println(round.(-Kd,digits=4))
println(round.(K₂,digits=4))

ErrorException: Addition between an array and a JuMP scalar is not supported: instead of `x + y`, do `x .+ y` for element-wise addition.

In [42]:
model = Model(Mosek.Optimizer);
n = 2
@variable(model, Σ[i=1:n, j=1:n], PSD)
@variable(model, Z[i=1:n, j=1:n], PSD)
@variable(model, L[i=1:1, j=1:n])
@objective(model, Min, tr(Z))
@constraint(model, Σ-1e-5*I >= 0, PSDCone())
# Z >= X^{-1}   
@constraint(model, [Z I; I Σ] >= 0, PSDCone())
# Lyupanov
@constraint(model, [Σ (A*Σ+B*L)'; (A*Σ+B*L) X] >= 0, PSDCone());
set_silent(model)
optimize!(model);

L, Σ, Z = value.(L), value.(Σ), value.(Z)
K = L * inv(Σ) 

1×2 Matrix{Float64}:
 -0.274775  -0.698508

([-1.4902018149978286e6 -3.842934205993686e7], [5.404510815271053e7 -1.7836972517953224e7; -1.7836972517953224e7 6.041490172454586e7], [2.327537352178023e-8 5.553688204393768e-9; 5.553688204393768e-9 2.120936661759748e-8])

1×2 Matrix{Float64}:
 -0.263149  -0.713783

In [19]:
eigen(A+B*K).values

2-element Vector{Float64}:
 0.325214498168043
 0.961002535440085